In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from haversine import haversine
print(datetime.datetime.now())
print(os.getcwd())


2020-05-04 12:35:33.877754
/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Competitor_conquest/BR_competitor_list_25_miles_from_BL_20200413


In [2]:
df_all_competitors=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Competitor_conquest/BL_competitor_list_with_geo_location_JL_2020-04-03.csv")
print(df_all_competitors.shape)
print(df_all_competitors['status'].unique())
df_all_competitors['nearest_BL_store']=np.nan
df_all_competitors['nearest_BL_dist']=np.nan

(3132, 11)
['Closed' 'impendingclose' 'open']


In [3]:
df_BL_stores=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",sep="|")
df_BL_stores=df_BL_stores[['location_id','latitude_meas','longitude_meas']]
df_BL_stores=df_BL_stores[~df_BL_stores['location_id'].isin([145,6990])]
df_BL_stores['coor']=df_BL_stores[['latitude_meas','longitude_meas']].values.tolist()
dict_BL_store_coor=df_BL_stores.set_index("location_id").to_dict()['coor']

In [4]:
for ind,row in df_all_competitors.iterrows():
    comp_location=(row['Google_lat'],row['Google_lng'])
    min_dist=np.inf
    nearest_store=np.nan
    for BL_id, BL_loc in dict_BL_store_coor.items():
        dist=haversine(comp_location,BL_loc,unit="mi")
        if dist<min_dist:
            min_dist=dist
            nearest_store=BL_id
    df_all_competitors.loc[ind,'nearest_BL_store']=nearest_store
    df_all_competitors.loc[ind,'nearest_BL_dist']=min_dist

print(df_all_competitors.shape)
df_all_competitors['nearest_BL_store']=df_all_competitors['nearest_BL_store'].astype(int)

df_all_competitors.head(2)
            

(3132, 13)


,store_name,address,city,state,zip_code,status,search_key,Google_Address,Google_lat,Google_lng,Google_zip,nearest_BL_store,nearest_BL_dist
0,Bed Bath & Beyond,700 meyerland plaza,houston,tx,77096,Closed,"700 meyerland plaza, houston, tx, 77096","700 Meyerland Plaza, Houston, TX 77096, USA",29.687731,-95.461461,77096.0,4195,2.891999
1,Bed Bath & Beyond,2306 highway 6 south,houston,tx,77077,Closed,"2306 highway 6 south, houston, tx, 77077","2306 S Texas 6, Houston, TX 77077, USA",29.738767,-95.644768,77077.0,1986,6.252690


In [5]:
excel_file=pd.ExcelFile("../BL_2020 Comp Store Closing and Opening 3.30.2020.xlsx")

def clean_df_add_key(df):
    for col in ['address','city','state','zip_code']:
        df[col]=df[col].fillna("")
        df[col]=df[col].apply(lambda x: x.strip().lower())

    df['search_key']=df[['address','city','state','zip_code']].values.tolist()
    df['search_key']=df['search_key'].astype(str)
    df['search_key']=df['search_key'].apply(lambda x: ", ".join(eval(x)))
    df=df.drop_duplicates()
    # print(df['store_name'].unique(),df.shape,df['search_key'].nunique())
    return(df)


# 1 closed
df_WL_1_closed=excel_file.parse("Wolf Furniture",dtype=str,skiprows=5)
df_WL_1_closed=df_WL_1_closed[['Address','City ','State','Zip','Status']].drop_duplicates()
df_WL_1_closed=df_WL_1_closed.rename(columns={"Address":"address",
                                                          "City ":"city",
                                                          "State":"state",
                                                          "Zip":"zip_code",
                                                          'Status':"status"})
df_WL_1_closed.insert(0,"store_name","Levin & Wolf")
df_WL_1_closed=df_WL_1_closed[df_WL_1_closed['status']!="New"] #remove the new
df_WL_1_closed=clean_df_add_key(df_WL_1_closed)
df_WL_1_closed['latitude']=39.3932386
df_WL_1_closed['longitude']=-77.4402846

# 2 open
df_WL_2_open=excel_file.parse("Levin & Wolf 2020 Stores",dtype=str)
df_WL_2_open=df_WL_2_open[['store_name','address','city','state','zip_code','latitude','longitude']].drop_duplicates()
df_WL_2_open=df_WL_2_open.rename(columns={"dba":"store_name"})
df_WL_2_open['status']="open"
df_WL_2_open=clean_df_add_key(df_WL_2_open)
df_WL_2_open['store_name']="Levin & Wolf"

df_WL=df_WL_1_closed.append(df_WL_2_open)
df_WL=df_WL[df_WL_2_open.columns.tolist()].reset_index()
df_WL['latitude']=df_WL['latitude'].astype(float)
df_WL['longitude']=df_WL['longitude'].astype(float)

del df_WL['index']
df_WL=df_WL.drop_duplicates("search_key")
df_WL['nearest_BL_store']=np.nan
df_WL['nearest_BL_dist']=np.nan

for ind,row in df_WL.iterrows():
    comp_location=(row['latitude'],row['longitude'])
    min_dist=np.inf
    nearest_store=np.nan
    for BL_id, BL_loc in dict_BL_store_coor.items():
        dist=haversine(comp_location,BL_loc,unit="mi")
        if dist<min_dist:
            min_dist=dist
            nearest_store=BL_id
    df_WL.loc[ind,'nearest_BL_store']=nearest_store
    df_WL.loc[ind,'nearest_BL_dist']=min_dist

print(df_WL.shape,df_WL['search_key'].nunique())
df_WL=df_WL[['store_name','address','city','state','zip_code','status','search_key','latitude','longitude','nearest_BL_store','nearest_BL_dist']]
df_WL['nearest_BL_store']=df_WL['nearest_BL_store'].astype(int)
df_WL.head(2)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(55, 11) 55


,store_name,address,city,state,zip_code,status,search_key,latitude,longitude,nearest_BL_store,nearest_BL_dist
0,Levin & Wolf,5830 ballenger creek pike,frederick,md,21703,Closed,"5830 ballenger creek pike, frederick, md, 21703",39.393239,-77.440285,5395,1.873197
1,Levin & Wolf,3742 brookwall drive,akron,oh,44333,open,"3742 brookwall drive, akron, oh, 44333",41.131590,-81.644194,1846,3.820870


In [6]:
df_list_1=df_WL[df_WL['nearest_BL_dist']<=25]
df_list_2=df_all_competitors[df_all_competitors['nearest_BL_dist']<=25]

df_list_selected=df_list_1.append(df_list_2)
df_list_selected=df_list_selected[df_list_selected['status']!="Closed"]
df_list_selected.shape,df_list_selected['search_key'].nunique()
df_list_selected=df_list_selected[df_all_competitors.columns.tolist()+['latitude','longitude']]

In [7]:
a=df_list_selected.groupby("search_key")['store_name'].nunique().to_frame().reset_index()
a=a[a['store_name']>=2]['search_key'].tolist()
df_list_selected[df_list_selected['search_key'].isin(a)]

,store_name,address,city,state,zip_code,status,search_key,Google_Address,Google_lat,Google_lng,Google_zip,nearest_BL_store,nearest_BL_dist,latitude,longitude
316,Bed Bath & Beyond,571 boston turnpike,shrewsbury,ma,01545,open,"571 boston turnpike, shrewsbury, ma, 01545","571 Boston Turnpike, Shrewsbury, MA 01545, USA",42.278105,-71.713127,1545.0,4691,8.332953,NaN,NaN
349,Bed Bath & Beyond,1327 george dieter drive,el paso,tx,79936,open,"1327 george dieter drive, el paso, tx, 79936","1327 George Dieter Dr, El Paso, TX 79936, USA",31.724602,-106.304349,79936.0,1926,1.830350,NaN,NaN
351,Bed Bath & Beyond,550 boardman poland road,youngstown,oh,44512,open,"550 boardman poland road, youngstown, oh, 44512","550 Boardman Poland Rd, Youngstown, OH 44512, USA",41.025541,-80.645038,44512.0,1466,0.485355,NaN,NaN
623,Bed Bath & Beyond,6090 garners ferry road,columbia,sc,29209,open,"6090 garners ferry road, columbia, sc, 29209","6090 Garners Ferry Rd, Columbia, SC 29209, USA",33.982086,-80.967941,29209.0,1012,0.744625,NaN,NaN
1126,Macy's,1 worcester road,framingham,ma,01701,open,"1 worcester road, framingham, ma, 01701","1 Worcester Rd, Framingham, MA 01701, USA",42.301714,-71.395061,1701.0,5194,3.448226,NaN,NaN
1857,Pier 1,1 worcester road,framingham,ma,01701,impendingclose,"1 worcester road, framingham, ma, 01701","1 Worcester Rd, Framingham, MA 01701, USA",42.301714,-71.395061,1701.0,5194,3.448226,NaN,NaN
2111,Pier 1,1000 turtle creek dr,hattiesburg,ms,39402,open,"1000 turtle creek dr, hattiesburg, ms, 39402","1000 Turtle Creek Dr, Hattiesburg, MS 39402, USA",31.325503,-89.375479,39402.0,5104,1.027441,NaN,NaN
2455,Pier 1,571 boston turnpike,shrewsbury,ma,01545,open,"571 boston turnpike, shrewsbury, ma, 01545","571 Boston Turnpike, Shrewsbury, MA 01545, USA",42.278105,-71.713127,1545.0,4691,8.332953,NaN,NaN
2508,Pier 1,550 boardman poland road,youngstown,oh,44512,open,"550 boardman poland road, youngstown, oh, 44512","550 Boardman Poland Rd, Youngstown, OH 44512, USA",41.025541,-80.645038,44512.0,1466,0.485355,NaN,NaN
2605,Pier 1,6090 garners ferry road,columbia,sc,29209,open,"6090 garners ferry road, columbia, sc, 29209","6090 Garners Ferry Rd, Columbia, SC 29209, USA",33.982086,-80.967941,29209.0,1012,0.744625,NaN,NaN


In [8]:
writer=pd.ExcelWriter("./BL_competitor_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_list_selected.to_excel(writer,"filtered_competitor_stores",index=False)
df_WL.to_excel(writer,"all_wolf_levin",index=False)
df_all_competitors.to_excel(writer,"all_other_5_comp",index=False)
writer.save()

In [9]:
df_list_selected.shape,df_list_selected['search_key'].nunique()

((2900, 15), 2894)

In [11]:
b=df_all_competitors[df_all_competitors['nearest_BL_dist']>25]
b=b[b['search_key'].isin(df_list_selected['search_key'].tolist())]
b.shape

(0, 13)